# Формирования таблицы фактов 'tenders' и 'products' для базы данных.

### 1. Подготовка

In [13]:

import time
import json
import datetime

import requests as rq
import pandas as pd
import numpy as np
import tqdm

from sqlalchemy import create_engine


### 2. Формирование переменных для подключения

In [ ]:

TOKEN = 'Bearer '
API = '/api/tenders/get'
URL = 'https://tenderplan.ru' + API

headers = {
    'Authorization': TOKEN,
    'Accept': 'application/json'
}

engine_pet = create_engine('mysql+mysqlconnector://'':''@''/tender_project')


### 3. Формирую переменную со списокм тендеров.

Для этого подтягиваю из временных таблиц сформированных в других скриптах нужные данные.

In [3]:

df_dict_tenders = pd.read_sql(
    'SELECT * FROM tmp_dict_tenders', 
    con = engine_pet, 
    index_col = '_id'
)

lst_tendrs = df_dict_tenders.index.to_list()


### 4. Формирование функции get запроса для цилка и функции преобразования времени

In [4]:

if __name__ == '__main__':
    def f_lst_tender(id_):
        '''Функция гет запроса принимающая ID тендера и выводит get запрос'''
        response = rq.get(
            URL + f'?id={str(id_)}',
            headers = headers
        ).json()
        return response
    
def f_date_fr_time(x):
    '''Принимает UNIX-время и возвращате дату в формате datetime'''
    return datetime.datetime.fromtimestamp(x // 1000)


### 4. Цикл формирует список с нужными ключами словаря и данными

In [6]:

start_time = time.time()
lst_tender_lst_dict = []

for x in tqdm.tqdm(lst_tendrs):
    get_tender = f_lst_tender(x)
    df_tenders_fact = {
        '_id': get_tender.get('_id'),
        'multilot': get_tender.get('multilot'),
        'attachments': get_tender.get('attachments'),
        'guaranteeApp': get_tender.get('guaranteeApp'),
        'guaranteeContract': get_tender.get('guaranteeContract'),
        'platform': get_tender.get('platform'),
        'products': get_tender.get('products')
    }
    lst_tender_lst_dict.append(df_tenders_fact)
    time.sleep(0.08)
    
end_time = time.time()
df_tenders_fact = lst_tender_lst_dict

del lst_tender_lst_dict
del get_tender


100%|██████████| 5795/5795 [29:31<00:00,  3.27it/s] 


### 5. Формирую два DataFrame: 

- DataFrame 'garant' дополняющий словарь тендеров атрибутами признака мультилот и суммами гарантий контракта;
- DataFrame ФЗ контрактов;
- DataFrame с платформами отогрышей тендеров.

И данный DataFrame joinю со справочником тендеров из временной таблицы

In [7]:

'''DataFrame с данными по мультилот и гарантиями контратка'''

df_garant = pd.DataFrame(
    data = df_tenders_fact, 
    columns = [
        '_id',
        'multilot', 
        'guaranteeApp',
        'guaranteeContract' 
    ]
).set_index('_id')

'''DataFrame с данными по форме ФЗ'''

df_attachments_ = (
    pd.DataFrame(
        data = df_tenders_fact, 
        columns = [
            '_id',
            'attachments'
        ]
    )
    .set_index('_id')
)

df_attachments = (
    pd.json_normalize(
        df_attachments_.explode('attachments')['attachments']
    )
    .set_index(df_attachments_.explode('attachments').index)
    .loc[
        :, 
        [
            'displayName', 
            'href', 
            'realName', 
            'size'
        ]
    ]
)

'''DataFrame с даннми по платформе отыгрышей тендеров'''

df_platform = pd.DataFrame(
    df_tenders_fact, 
    columns = [
        '_id',
        'platform'
    ]
).set_index('_id')

df_platform = pd.json_normalize(df_platform['platform']).set_index(df_platform.index)

df_garant = (
    df_garant
    .assign(
        **{
            'fz': df_attachments_.map((
                lambda x: x[0]['href']
                .split('/')[3]
                .removesuffix('fz')
            ))
        }
    )
    .join(df_platform, lsuffix='/garant', rsuffix='/platgorm')
    .join(df_dict_tenders, lsuffix='/garant', rsuffix='/dict')
)

'''DataFrame со спецификацией тендера'''

df_products = (
    pd.DataFrame(
        df_tenders_fact, 
        columns = [
            '_id',
            'products'
        ]
    )
    .set_index('_id')
    .explode('products')
)

lst_nm_pr = [
    'name',
    'price',
    'quantity',
    'unit'
]

df_products = (
    pd.json_normalize(df_products['products'])
    .set_index(df_products.index)
    .loc[:,lst_nm_pr]
    .dropna(subset = lst_nm_pr)
)

del df_attachments_
del df_platform
del lst_nm_pr
del df_dict_tenders


### 6. Загрузка данных во временную таблицу справочника тендеров (замена) и загрузка в новую временную таблицу таблицы фактов со спецификацией тендера

In [8]:

(
    df_garant
    .reset_index()
    .to_sql(
        'tmp_dict_tenders', 
        con = engine_pet, 
        if_exists = 'replace', 
        index = False
    )
)

time.sleep(1)

(
    df_products
    .reset_index()
    .to_sql(
        'tmp_products', 
        con = engine_pet, 
        if_exists = 'replace', 
        index = False
    )
)


-1

In [ ]:

print(df_garant.head())
print(df_products.head())
